In [94]:
!pip install beautifulsoup4
!pip install pdfplumber
import requests
from bs4 import BeautifulSoup
import pandas as pd
from lxml import etree
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import pdfplumber
!pip install SQLAlchemy
!pip install pymysql
import sqlalchemy as db

In [166]:
options= webdriver.ChromeOptions()
chrome_path = './chromedriver'

gastronomico = ['queijo','leite','prato', 'comida', 'culinária','café','restaurante', 'comer', 'doces', 'cachaça']
cultural =['museu', 'casarão', 'casa', 'histórico', 'histórica', 'sítios arqueológicos']
religioso = ['igreja', 'capela', 'basílica', 'procissão', 'católica', 'Nossa Senhora', 'catedral','Aleijadinho', 'Ataíde ']
aventura= ['cachoeira', 'cânions', 'cavernas', 'cachoeiras', 'rios', 'matas', 'rio', 'trekking', 'caminhada', 'rapel', 'escalada', 'trilhas', 'aventura','adrenalina', 'rafting']

url_caminho_dos_diamantes = 'https://institutoestradareal.com.br/caminhos/caminho-dos-diamantes/'
url_caminho_novo = 'https://institutoestradareal.com.br/caminhos/caminho-novo/'
url_caminho_velho = 'https://institutoestradareal.com.br/caminhos/caminho-velho/'
url_caminho_do_sabarabucu = 'https://institutoestradareal.com.br/caminhos/caminho-do-sabarabucu/'

  ## Caminhos

In [165]:
caminhos=['Caminho dos Diamantes', 'Caminho Novo', 'Caminho Velho', 'Caminho do Sabarabuçu']
codigo_caminho=['01', '02', '03', '04']
inicio=['Diamantina','Ouro Preto','Ouro Preto', 'Morro Vermelho']
fim=['Ouro Preto','Porto Estrela', 'Paraty','Honório Bicalho']
caminhos_df = pd.DataFrame([caminhos,codigo_caminho,inicio,fim]).T
caminhos_df.columns = ["Caminho", 'Código do caminho', 'Início', 'Fim']
caminhos_df

Caminho Código do caminho          Início              Fim
0  Caminho dos Diamantes                01      Diamantina       Ouro Preto
1           Caminho Novo                02      Ouro Preto    Porto Estrela
2          Caminho Velho                03      Ouro Preto           Paraty
3  Caminho do Sabarabuçu                04  Morro Vermelho  Honório Bicalho

In [167]:
def conecta_site (url):
    """Conecta-se à página e transforma o conteúdo em sopa para posteriores pesquisas
    Attr. URL do caminho
    return caminho xpath para a sopa (soup_xpath)"""
    get_html = requests.get(url)
    html_content = get_html.content
    soup = BeautifulSoup(html_content, "lxml")
    soup_xpath = etree.HTML(soup.prettify())
    return soup_xpath

In [168]:
def dados_cidades (links, roteiro):
    for link in links:
        url = link
        get_html = requests.get(url)
        html_content = get_html.content
        soup = BeautifulSoup(html_content, "lxml")
        soup_xpath = etree.HTML(soup.prettify())
        caminho.append(roteiro)
        cidade.append(soup_xpath.xpath('/html/body/div/div/div/div/article/h1')[0].text.strip())
        estado.append(url.split('-')[-1].upper())
        pontos=[]
        for ponto_turistico in soup_xpath.xpath('/html/body/div[1]/section[2]/div/div/div/div/h5/a'):
            pontos.append(ponto_turistico.text.strip())
        try:
            pontos_turisticos[soup_xpath.xpath('/html/body/div/div/div/div/article/h1')[0].text.strip()] = (pontos)
        except:
            continue
        try:
            float(soup_xpath.xpath('//*[@id="__next"]/div/div/div/article/ul/li/span/text()')[3].strip())
            habitantes.append(soup_xpath.xpath('//*[@id="__next"]/div/div/div/article/ul/li/span/text()')[3].strip())
        except:
            habitantes.append(soup_xpath.xpath('//*[@id="__next"]/div/div/div/article/ul/li/span/text()')[4].strip())
        texto = soup.select('article')[0].text
        ecoturismo_e_turismo_de_aventura.append(verifica_roteiro(texto, aventura))
        turismo_religioso.append(verifica_roteiro(texto, religioso))
        turismo_cultural.append(verifica_roteiro(texto, cultural))
        turismo_gastronomico.append(verifica_roteiro(texto, gastronomico))   

In [169]:
def verifica_roteiro(texto, tipo_de_turismo):
    """Verifica se as palavras-chave para cada tipo de turismo
    estão na descrição da cidade feita pelo Instituto Estrada Real"""
    for palavra in tipo_de_turismo:
        if palavra.lower() in texto.lower():
            return True
    return False

## Caminho dos Diamantes

In [173]:
caminho=[]
cidade =[]
estado =[]
habitantes =[]
turismo_gastronomico = []
turismo_cultural= []
turismo_religioso= []
ecoturismo_e_turismo_de_aventura= []

pontos_turisticos ={}

#Caminho dos Diamantes
soup_xpath_diamantes = conecta_site(url_caminho_dos_diamantes)
caminho_dos_diamantes=[cidade.attrib['value'] for cidade in soup_xpath_diamantes.xpath('//*[@id="__next"]/section[2]/div/div/select/option')]
links_cidades_cdd =['https://institutoestradareal.com.br'+cidade.split('(')[0] for cidade in caminho_dos_diamantes]
links_cidades_cdd.remove('https://institutoestradareal.com.br')
dados_cidades (links_cidades_cdd, '01')
#Caminho novo
soup_xpath_novo = conecta_site(url_caminho_novo)
caminho_novo=[cidade.attrib['value'] for cidade in soup_xpath_novo.xpath('//*[@id="__next"]/section[2]/div/div/select/option')]
links_cidades_cn =['https://institutoestradareal.com.br'+cidade.split('(')[0] for cidade in caminho_novo]
links_cidades_cn.remove('https://institutoestradareal.com.br')
dados_cidades (links_cidades_cn, '02')
#Caminho Velho
soup_xpath_velho = conecta_site(url_caminho_velho)
caminho_velho=[cidade.attrib['value'] for cidade in soup_xpath_velho.xpath('//*[@id="__next"]/section[2]/div/div/select/option')]
links_cidades_cv =['https://institutoestradareal.com.br'+cidade.split('(')[0] for cidade in caminho_velho]
links_cidades_cv.remove('https://institutoestradareal.com.br')
dados_cidades (links_cidades_cv, '03')
#Caminho do Sabarabuçu
soup_xpath_sabarabucu = conecta_site(url_caminho_do_sabarabucu)
caminho_do_sabarabucu=[cidade.attrib['value'] for cidade in soup_xpath_sabarabucu.xpath('//*[@id="__next"]/section[2]/div/div/select/option')]
links_cidades_cs =['https://institutoestradareal.com.br'+cidade.split('(')[0] for cidade in caminho_do_sabarabucu]
links_cidades_cs.remove('https://institutoestradareal.com.br')
dados_cidades (links_cidades_cs, '04')

## Data Frame

In [174]:
estrada_real_df = pd.DataFrame([caminho, estado, cidade, habitantes, turismo_gastronomico, turismo_cultural, turismo_religioso, ecoturismo_e_turismo_de_aventura]).T
estrada_real_df.columns=['Caminho','Estado','Cidade','Habitantes', 'Turismo Gastronômico', 'Turismo Cultural', 'Turismo Religioso', 'Ecoturismo e Turismo de Aventura']
estrada_real_df

Caminho Estado             Cidade Habitantes Turismo Gastronômico  \
0        01     MG            Acaiaca      3.924                 True   
1        01     MG        Alvinópolis     15.212                False   
2        01     MG  Alvorada de Minas      3.548                 True   
3        01     MG    Barão de Cocais     28.432                False   
4        01     MG        Barra Longa      5.250                 True   
..      ...    ...                ...        ...                  ...   
244      04     MG     Morro Vermelho          0                False   
245      04     MG          Nova Lima     87.391                 True   
246      04     MG            Raposos     15.345                False   
247      04     MG          Rio Acima      9.095                False   
248      04     MG             Sabará    126.219                 True   

    Turismo Cultural Turismo Religioso Ecoturismo e Turismo de Aventura  
0               True              True                             True  
1              False             False                             True  
2               True             False                             True  
3               True             False                             True  
4               True              True                             True  
..               ...               ...                              ...  
244            False              True                             True  
245             True              True                             True  
246             True              True                             True  
247             True              True                             True  
248             True              True                             True  

[249 rows x 8 columns]

## Pontos Turísticos

In [46]:
pontos_turisticos_df= pd.DataFrame.from_dict(pontos_turisticos, orient='index')
pontos_turisticos_df = pontos_turisticos_df.T.melt().dropna()
pontos_turisticos_df

variable                                      value
0               Acaiaca                        Estação Ferroviária
8           Alvinópolis  Igreja Matriz de Nossa Senhora do Rosário
16    Alvorada de Minas                       Cachoeira da Campina
17    Alvorada de Minas                Igreja Matriz Santo Antônio
18    Alvorada de Minas                       Mirante da Escadinha
...                 ...                                        ...
1955             Sabará         Festival do Ora-pro-nóbis - Sabará
1956             Sabará           Igreja de Nossa Senhora do Carmo
1957             Sabará          Igreja Nossa Senhora da Conceição
1958             Sabará            Igreja Nossa Senhora das Mercês
1959             Sabará                  Igreja Nossa Senhora do Ó

[604 rows x 2 columns]

## Meso e microrregiões

In [47]:
meso_e_micro_mg = 'https://pt.wikipedia.org/wiki/Lista_de_mesorregi%C3%B5es_e_microrregi%C3%B5es_de_Minas_Gerais'

In [48]:
get_html = requests.get(meso_e_micro_mg)
html_content = get_html.content
soup = BeautifulSoup(html_content, "lxml")

In [49]:
#tabela meso e microrregiões:
tabela_micro_meso_mg=soup.find_all('table')[0]
tabela_micro_meso_mg = tabela_micro_meso_mg.prettify()
tabela_micro_meso_mg= pd.read_html(tabela_micro_meso_mg)[0]
tabela_micro_meso_mg = tabela_micro_meso_mg.drop(['Área  (km²/2002)  [  4  ]',
       'Número de  municípios', 'Localização'], axis =1)
tabela_micro_meso_mg.columns=['Mesorregião', 'codigo_meso', 'Microrregião', 'codigo_micro']
#normatização de códigos
lista_codigo_micro=[]
for codigo_micro in tabela_micro_meso_mg['codigo_micro']:   
    lista_codigo_micro.append('mi'+str(codigo_micro))
tabela_micro_meso_mg['codigo_micro'] = lista_codigo_micro
lista_codigo_meso=[]
for codigo_meso in tabela_micro_meso_mg['codigo_meso']:   
    lista_codigo_meso.append('me'+str(codigo_meso))
tabela_micro_meso_mg['codigo_meso'] = lista_codigo_meso

In [50]:
#tabelas microrregiões e municípios:
len(soup.find_all('table'))
tabelas_micro_municipio_mg=[]
for tabela in range(1,len(soup.find_all('table'))):
    tabela_micro_municipio_mg = (soup.find_all('table')[tabela])
    tabela_micro_municipio_mg= tabela_micro_municipio_mg.prettify()
    tabela_micro_municipio_mg= pd.read_html(tabela_micro_municipio_mg)[0]
    tabelas_micro_municipio_mg.append((tabela_micro_municipio_mg))
tabelas_micro_municipio_mg= pd.concat(tabelas_micro_municipio_mg)
tabelas_micro_municipio_mg = pd.DataFrame(tabelas_micro_municipio_mg)
tabelas_micro_municipio_mg=tabelas_micro_municipio_mg.drop(['Localização', 'População (IBGE 2010)', 'População (IBGE 2010).1',
       'v  d  e  Mesorregiões e microrregiões  do  Brasil',
       'v  d  e  Mesorregiões e microrregiões  do  Brasil.1',
       'v  d  e  Minas Gerais', 'v  d  e  Minas Gerais.1'], axis =1)
tabelas_micro_municipio_mg.columns=['Microrregião', 'codigo_micro','Município']
tabelas_micro_municipio_mg=tabelas_micro_municipio_mg.dropna()
#normatização de códigos
lista_codigo=[]
for codigo in tabelas_micro_municipio_mg['codigo_micro']:   
    lista_codigo.append('mi'+(str(codigo)).split('.')[0])
tabelas_micro_municipio_mg['codigo_micro'] = lista_codigo
codigo_municipio =['c'+ str(municipio) for municipio in range(1,len(tabelas_micro_municipio_mg['Município'])+1)]
tabelas_micro_municipio_mg.insert(2, 'codigo_municipio', codigo_municipio)

# Hoteis Pet Friendly

In [109]:
cidades_maiores =[]
for municipio in tabela_micro_meso_mg['Microrregião']:
    if municipio in cidade:
        cidades_maiores.append(municipio)
cidades_maiores

['Diamantina',
 'Conceição do Mato Dentro',
 'Itabira',
 'Ouro Preto',
 'Conselheiro Lafaiete',
 'Guanhães',
 'São Lourenço',
 'Andrelândia',
 'Lavras',
 'Barbacena',
 'Ponte Nova',
 'Juiz de Fora']

In [131]:
#driver = webdriver.Chrome(executable_path=chrome_path)
hoteis={}
chrome_prefs = {}
options.experimental_options["prefs"] = chrome_prefs
chrome_prefs["profile.default_content_settings"] = {"javascript": 2}
chrome_prefs["profile.managed_default_content_settings"] = {"javascript": 2}
driver = webdriver.Chrome('C:/Users/Tete/Desktop/chromedriver',options=options)
for municipio in cidade:  
    hotel=[]
    municipio.lower().replace(" ", "-").replace("é", "e").replace("ó", "o").replace("í", "i").replace("ã", "a")
    driver.get(f'https://www.booking.com/searchresults.pt-br.html?label=gen173nr-1FCAEoggI46AdIM1gEaCCIAQGYAS24ARfIAQzYAQHoAQH4AQuIAgGoAgO4AprxhpMGwAIB0gIkMDUyZDJkYzMtOTVmMi00YTdiLTg1NDItN2ZlYTZjYzM2NGU12AIG4AIB&sid=0eb892a9bd794d3189d0cee1766518ed&aid=304142&ss={municipio}&ssne={municipio}&ssne_untouched={municipio}&lang=pt-br&src=searchresults&checkin=2022-06-04&checkout=2022-06-05&group_adults=2&no_rooms=1&group_children=0&sb_travel_purpose=leisure&nflt=hotelfacility%3D4')
    driver.maximize_window()
    time.sleep(1)
    local = driver.find_element('xpath','//*[@id="right"]/div/div/div/div/h1').text.split(':')[0]
    try:
        nome_do_hotel = driver.find_elements('xpath','/html/body/div[2]/div/div[3]/div[1]/div[1]/div[4]/div[2]/div/div/div/div/div[6]/div/div/div/div/div/div/div/div/div/h3/a/div')
        for acomodacao in nome_do_hotel:
            acomodacao = acomodacao.text.strip()
            if acomodacao != 'Abre numa nova janela':
                hotel.append(acomodacao)
                hoteis[local]=(hotel)
            else:
                continue
    except:
        continue            
    
    #driver.close()

C:\Users\Tete\AppData\Local\Temp/ipykernel_22232/674395203.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('C:/Users/Tete/Desktop/chromedriver',options=options)


In [132]:
hoteis

{'0 propriedade tem disponibilidade neste destino e arredores': ['Pousada Villa Real',
  'Chale Mineiro Hostel & Pousada',
  'ibis budget Belo Horizonte Afonso Pena',
  'Tryp by Wyndham Belo Horizonte Savassi',
  'ibis Belo Horizonte Afonso Pena',
  'Holiday Inn Belo Horizonte Savassi, an IHG Hotel',
  'Royal Golden Hotel - Savassi',
  'Vila Santê',
  'Aconchego dos schreiber',
  'ibis budget BH Savassi',
  'ibis Belo Horizonte Savassi',
  'ibis Belo Horizonte Liberdade',
  'Hotel Real de Caeté',
  'Novotel BH Savassi',
  'Rio Jordao Hotel BH',
  'eSuites Savassi Toscanini',
  'Max Savassi Apart Service',
  'Royal Design Savassi Hotel',
  'Royal Boutique Savassi Hotel',
  'Brasil Palace Hotel',
  'Nacional Inn Belo Horizonte',
  'Boulevard Plaza',
  'Boulevard Park',
  'Flat no Ianelli',
  'Boulevard Express'],
 'Barão de Cocais': ['Impar Suites Barao de Cocais',
  'ESTÂNCIA JARDIM SECRETO',
  'Pousada das Nascentes',
  'Hotel pousada & Eventos Cassino',
  'Hotel Real de Caeté',
  'Pou

In [139]:
hoteis_pet_friendly_df= pd.DataFrame.from_dict(hoteis, orient='index')
hoteis_pet_friendly_df = hoteis_pet_friendly_df.T.melt().dropna()

In [ ]:
titulo =[]
for hotel in hoteis['hotel']:
    titulo.append(hotel.title())
hoteis['hotel'] = titulo

## My SQL Workbench

In [98]:
db_server='pymysql'
user='root'
db_port = '3306'
password = "*********"
ip = 'localhost'
db_name = 'estrada_real'
engine = db.create_engine(f'mysql+{db_server}://{user}:{password}@{ip}:{db_port}/{db_name}?charset=utf8')
conn = engine.connect()
#conn.close()


In [99]:
caminhos_df.to_sql('caminhos', conn, if_exists='replace', index=False)
estrada_real_df.to_sql('cidades', conn, if_exists='replace', index=False)
pontos_turisticos_df.to_sql('pontos_turisticos', conn, if_exists='replace', index=False)
tabela_micro_meso_mg.to_sql('micro_mesorregioes_mg', conn, if_exists='replace', index=False)
tabelas_micro_municipio_mg.to_sql('micro_municipios_mg', conn, if_exists='replace', index=False)
hoteis_pet_friendly_df.to_sql('hoteis_pet_friendly', conn, if_exists='replace', index=False)